In [22]:
from pymatgen import MPRester, Element, Composition
from pymatgen.phasediagram.maker import PhaseDiagram
from pymatgen.phasediagram.entries import PDEntry
from pymatgen.phasediagram.analyzer import PDAnalyzer
from pymatgen.analysis.reaction_calculator import ComputedReaction
import matplotlib.pyplot as plt
import pandas as pnds
import sys

chempots = [] #critical chemical potentials at every transition point
evolutions = [] 
phases = []
enthalpies = []
reactions = []
yvec = []

formula = 'Li13Al3Ti17P30O120'
elem = 'O'
els = sorted(Composition(formula).keys())
chsys = [x.symbol for x in els]

with MPRester() as m:
    entries = m.get_entries_in_chemsys(chsys)
    mpentry = m.get_entries(formula) #gets all MP-entries associated with given composition

pd = PhaseDiagram(entries)
pda = PDAnalyzer(pd)

if len(mpentry) == 0: #composition is metastable, so force it to lie on the convex hull
    compound = PDEntry(Composition(formula), 0)
    e0 = pda.get_e_above_hull(compound)
    compound = PDEntry(Composition(formula), -(e0*Composition(formula).num_atoms))
else: #otherwise return the most stable entry in the list of MP-entries. This entry can be metastable.
            #in the future, may want to include a shift of "e-above-hull" to set this compound on the x-axis
    mpentry = sorted(mpentry, key=lambda e: e.energy_per_atom)
    compound = mpentry[0]


evolution_profile = pda.get_element_profile(Element(elem), Composition(formula))
ref = evolution_profile[0]['element_reference'].energy_per_atom
    
reac = [compound]
reac.append(evolution_profile[0]['element_reference'])
    
for stage in evolution_profile:
    chempots.append(stage['chempot'] - ref)
    evolutions.append(stage['evolution'])
    namelist = []
    for entry in stage['entries']:
        namelist.append(entry.name)
    phases.append(namelist)
    
yshift = 0  #initialize y-shift
window = False #assume the compound has no stability window at first
for stage in evolution_profile:
    rxn = ComputedReaction(reac, stage['entries']) #calculate reaction enthalpy
    rxn.normalize_to(Composition(compound.name))
    enthalpies.append(rxn.calculated_reaction_energy)
    reactions.append(str(rxn))
    if abs(stage['evolution']) < 0.0001: #establish presence of stability window
        window = True
        yshift = stage['evolution']*stage['chempot'] + rxn.calculated_reaction_energy
    
if not window: #if there is no stability window, check if it is metastable
    ehull = pda.get_e_above_hull(compound)
    if ehull > 0: #the compound is metastable
        yshift = -(ehull*comp_entry.num_atoms) #set y-shift to the e above hull for the formula unit
            
for i in range(len(chempots)): #manipulate and shift data for graphs
    ynew = (evolutions[i]*(-chempots[i]) + enthalpies[i]) - yshift
    yvec.append(ynew)            
    
pnds.options.display.float_format = '{:.3f}'.format
table1 = pnds.DataFrame({r'$\mu_O$* (eV)':chempots,
                      'Phase Equilibria':phases,
                     r'$\Delta H_{rxn}$ at $\mu_O$ = 0.0':enthalpies,
                    r'$\Delta H_{rxn}$ at $\mu_O$*':yvec,
                        'Reaction':reactions})
pnds.set_option('max_colwidth', 100)
display(table1)

,$\Delta H_{rxn}$ at $\mu_O$ = 0.0,$\Delta H_{rxn}$ at $\mu_O$*,$\mu_O$* (eV),Phase Equilibria,Reaction
0,-0.000,0.000,0.000,"[AlPO4, LiTi2(PO4)3, Li3PO4]",Li13Ti17Al3(PO4)30 -> 8.5 LiTi2(PO4)3 + 1.5 Li3PO4 + 3 AlPO4
1,58.645,-0.000,-3.910,"[P, AlPO4, LiTiPO5, LiTi2(PO4)3]",Li13Ti17Al3(PO4)30 -> 7.5 O2 + 9 LiTiPO5 + 4 LiTi2(PO4)3 + 3 AlPO4 + 6 P
2,137.051,-0.160,-3.920,"[P, AlPO4, LiTiPO5, TiO2]",Li13Ti17Al3(PO4)30 -> 17.5 O2 + 13 LiTiPO5 + 3 AlPO4 + 4 TiO2 + 14 P
3,222.455,-0.908,-3.942,"[P, AlPO4, Li3PO4, TiO2]",Li13Ti17Al3(PO4)30 -> 28.33 O2 + 4.333 Li3PO4 + 3 AlPO4 + 17 TiO2 + 22.67 P
4,314.205,-6.921,-4.048,"[AlPO4, TiO2, TiP2, Li3PO4]",Li13Ti17Al3(PO4)30 -> 39.67 O2 + 4.333 Li3PO4 + 3 AlPO4 + 11.33 TiP2 + 5.667 TiO2
5,359.802,-8.770,-4.071,"[TiO2, LiAl5O8, TiP2, Li3PO4]",Li13Ti17Al3(PO4)30 -> 45.27 O2 + 0.6 LiAl5O8 + 4.133 Li3PO4 + 12.93 TiP2 + 4.067 TiO2
6,393.440,-14.629,-4.136,"[TiP, LiAl5O8, TiP2, Li3PO4]",Li13Ti17Al3(PO4)30 -> 49.33 O2 + 0.6 LiAl5O8 + 4.133 Li3PO4 + 8.133 TiP + 8.867 TiP2
7,402.207,-39.066,-4.384,"[TiP, LiAlO2, TiP2, Li3PO4]",Li13Ti17Al3(PO4)30 -> 50.33 O2 + 3 LiAlO2 + 3.333 Li3PO4 + 7.333 TiP + 9.667 TiP2
8,432.869,-76.757,-4.758,"[TiP, LiAlO2, Li3P7, Li3PO4]",Li13Ti17Al3(PO4)30 -> 53.56 O2 + 3 LiAlO2 + 1.611 Li3P7 + 1.722 Li3PO4 + 17 TiP
9,465.963,-81.682,-4.804,"[TiP, LiAlO2, Li3P7, LiP]",Li13Ti17Al3(PO4)30 -> 57 O2 + 3 LiAlO2 + 0.75 Li3P7 + 7.75 LiP + 17 TiP
